In [ ]:
from backend.database import Database
from backend.agents import RewriterAgent, GeneratorAgent, ValidatorAgent
import pandas as pd
from time import sleep

In [3]:
database = Database()
rewriter = RewriterAgent()
generator = GeneratorAgent()
validator = ValidatorAgent()

In [6]:
eval_df = pd.read_csv(r'..\evaluation.csv')
eval_df.head()

,question,difficulty
0,List all actors' first and last names.,Easy
1,Show the titles of all films in the database.,Easy
2,Get the names of all cities.,Easy
3,List all categories available for films.,Easy
4,Show the first name and last name of all custo...,Easy


In [ ]:
generated_queries = []
explanations = []
retrieved_datasets = []

In [ ]:
for question in eval_df['question']:
    try:
        rewriter_response = rewriter.run({'question': question})
        sleep(0.1)
        generator_response = generator.run(rewriter_response)
        sleep(0.1)
        validator_response = validator.run(generator_response)

        sql_query = validator_response['query']
        explanation = validator_response['explanation']
        if sql_query is not None:
            result_df = database.get_data(sql_query)
        else:
            result_df = None

        generated_queries.append(sql_query)
        explanations.append(explanation)
        try: 
            retrieved_datasets.append(result_df.to_dict(orient='records'))  
        except: 
            retrieved_datasets.append("Couldn't Retrive")  
            
    except Exception as e:
        generated_queries.append(None)
        explanations.append(str(e))
        retrieved_datasets.append(None)


In [ ]:

eval_df['generated_query'] = generated_queries
eval_df['explanation'] = explanations
eval_df['retrieved_dataset'] = retrieved_datasets

eval_df.to_csv('evaluation_report.csv', index=False)

